In [1]:
import os
from pathlib import Path

import av
import numpy as np
import torch
from moviepy import editor
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from pytorchvideo.data.encoded_video import EncodedVideo
from transformers import (
    VideoMAEForVideoClassification,
    VideoMAEImageProcessor,
    VideoMAEModel,
)

np.random.seed(0)

## **Helper Functions**

In [2]:
# Got them from: https://huggingface.co/docs/transformers/model_doc/videomae#transformers.VideoMAEForVideoClassification.forward.example


def read_video_pyav(container, indices):
    """
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    """
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [3]:
def split_video_to_clips(video_path: str, output_folder: str, window: int) -> None:
    # Load video and get its duration.
    clip = editor.VideoFileClip(video_path)
    duration = clip.duration
    del clip
    
    start_ = 0
    end_ = window
    
    os.makedirs(output_folder, exist_ok=True)
    while end_ < duration:
        ffmpeg_extract_subclip(
            video_path,
            start_,
            end_,
            targetname=os.path.join(output_folder, f"{start_}_{end_}.mp4"),
        )
        start_ += window
        end_ += window  # TODO: There is a possibility to miss the last clip. Maybe we should set an upper bound.

## **Split a video into clips**

In [4]:
video_path = "test_video.mp4"
saveto = os.path.basename(video_path)
saveto = Path(saveto).stem

split_video_to_clips(video_path=video_path, output_folder=saveto, window=10)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


## **Extract audio from a clip**

In [14]:
#Load the Video
video = editor.VideoFileClip("test_video.mp4")

#Extract the Audio
audio = video.audio

#Export the Audio
audio.write_audiofile("audio.mp3")

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


## **Extract a feature vector from a video**

In [5]:
processor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
# model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")

Some weights of the model checkpoint at MCG-NJU/videomae-base-finetuned-kinetics were not used when initializing VideoMAEModel: ['fc_norm.bias', 'classifier.weight', 'fc_norm.weight', 'classifier.bias']
- This IS expected if you are initializing VideoMAEModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoMAEModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = "test_video.mp4"
container = av.open(file_path)

# sample 16 frames
indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

# prepare video for the model
inputs = processor(list(video), return_tensors="pt")

In [7]:
with torch.no_grad():
    outputs = model(**inputs)
    # logits = outputs.logits

In [8]:
outputs.last_hidden_state.shape

torch.Size([1, 1568, 768])

In [12]:
video_features = outputs.last_hidden_state.squeeze(dim=0)[0, :]

In [13]:
video_features.shape

torch.Size([768])